# Informed Search

Also known as "heuristic" search, because the search is informed by an
estimate of the total path cost through each node, and the next
unexpanded node with the lowest estimated cost is expanded next. 

    At some intermediate node, the 
      estimated cost of the solution path =
          the sum of the step costs so far from the start node to this node
             +
          an estimate of the sum of the remaining step costs to a goal

Let's label these as

   * $f(n) =$ estimated cost of the solution path through node $n$
   * $g(n) =$ the sum of the step costs so far from the start node to this node
   * $h(n) =$ an estimate of the sum of the remaining step costs to a goal

*heuristic function*: $h(n) =$ estimated cost of the cheapest path from state at node $n$ to a goal state.

<img src='astarfgh.png'>

Should we explore under Node a or b?

<img src='astarfgh2.png'>

# A* algorithm

## Non-recursive

So, now you know enough python to try to implement A\*, at least a non-recursive form.  Start with your graph search algorithm from Assignment 1.  Modify it so that the next node selected is based on its `f` value.

For a given problem, define `startState`, `actionsF`, `takeActionF`, `goalTestF`, and a heuristic function `hF`.  'actionsF' must return valid actions paired with the single step cost, and `takeActionF` must return the pair containing the new state and the path cost so far from the start state to the new state.  We can use the `Node` class to hold instances of nodes.  However, since this is not a recursive algorithm, `Node` must be extended to include the node's parent node, to be able to generate the solution path once the search finds the goal.

Now the A* algorithm can be written as follows
  * Initialize `expanded` to be an empty dictionary
  * Initialize `unExpanded` to be a list containing the startState node.  Its `h` value is calculated using `hF`, its `g` value is 0, and its `f` value is `g+h`.
  * If `startState` is the `goalState`, return the list containing just `startState` and its `f` value to show the cost of the solution path.
  * Repeat the following steps while `unExpanded` is not empty:
    * Pop from the front of `unExpanded` to get the best (lowest f value) node to expand.
    * Generate the `children` of this `node`.
    * Update the `g` value of each child by adding the action's single step cost to this node's `g` value.
    * Calculate `hF` of each child.
    * Set `f = g + h` of each child.
    * Add the node to the `expanded` dictionary, indexed by its state.
    * Remove from `children` any nodes that are already either in `expanded` or `unExpanded`, unless the node in `children` has a lower f value.
    * If `goalState` is in `children`:
      * Build the solution path as a list starting with `goalState`.  
      * Use the parent stored with each node in the `expanded` dictionary to construct the path.
      * Reverse the solution path list and return it.
    * Insert the modified `children` list into the `unExpanded`   list and ** sort by `f` values.**

## Recursive

Our authors provide the Recursive Best-First Search algorithm, which
is A\* in a recursive, iterative-deepening form, where depth is now
given by the $f$ value.  Other differences from just
iterative-deepening A\* are:
  - depth-limit determined by $f$ value of best alternative to node being explored, so will stop when alternative at the node's level looks better;
  - $f$ value of a node is replaced by best $f$ value of its children, so any future decision to try expanding this node again is more informed.

It is a bit difficult to translate their pseudo-code into python.  Here is my version.  Let's step through it.

In [1]:
%%writefile aStarSearch.py
# Recursive Best First Search (Figure 3.26, Russell and Norvig)
#  Recursive Iterative Deepening form of A*, where depth is replaced by f(n)

class Node:
    def __init__(self, state, f=0, g=0 ,h=0):
        self.state = state
        self.f = f
        self.g = g
        self.h = h
    def __repr__(self):
        return "Node(" + repr(self.state) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"

def aStarSearch(startState, actionsF, takeActionF, goalTestF, hF):
    h = hF(startState)
    startNode = Node(state=startState, f=0+h, g=0, h=h)
    return aStarSearchHelper(startNode, actionsF, takeActionF, goalTestF, hF, float('inf'))

def aStarSearchHelper(parentNode, actionsF, takeActionF, goalTestF, hF, fmax):
    if goalTestF(parentNode.state):
        return ([parentNode.state], parentNode.g)
    ## Construct list of children nodes with f, g, and h values
    actions = actionsF(parentNode.state)
    if not actions:
        return ("failure", float('inf'))
    children = []
    for action in actions:
        (childState,stepCost) = takeActionF(parentNode.state, action)
        h = hF(childState)
        g = parentNode.g + stepCost
        f = max(h+g, parentNode.f)
        childNode = Node(state=childState, f=f, g=g, h=h)
        children.append(childNode)
    while True:
        # find best child
        children.sort(key = lambda n: n.f) # sort by f value
        bestChild = children[0]
        if bestChild.f > fmax:
            return ("failure",bestChild.f)
        # next lowest f value
        alternativef = children[1].f if len(children) > 1 else float('inf')
        # expand best child, reassign its f value to be returned value
        result,bestChild.f = aStarSearchHelper(bestChild, actionsF, takeActionF, goalTestF,
                                            hF, min(fmax,alternativef))
        if result is not "failure":               #        g
            result.insert(0,parentNode.state)     #       / 
            return (result, bestChild.f)          #      d
                                                  #     / \ 
if __name__ == "__main__":                        #    b   h   
                                                  #   / \   
    successors = {'a': ['b','c'],                 #  a   e  
                  'b': ['d','e'],                 #   \         
                  'c': ['f'],                     #    c   i
                  'd': ['g', 'h'],                #     \ / 
                  'f': ['i','j']}                 #      f  
    def actionsF(s):                              #       \ 
        try:                                      #        j
            ## step cost of each action is 1
            return [(succ,1) for succ in successors[s]]
        except KeyError:
            return []
    def takeActionF(s,a):
        return a
    def goalTestF(s):
        return s == goal
    def h1(s):
        return 0
    start = 'a'
    goal = 'h'
    result = aStarSearch(start,actionsF,takeActionF,goalTestF,h1)
    print('Path from a to h is', result[0], 'for a cost of', result[1])

Writing aStarSearch.py


Running this shows

In [2]:
run aStarSearch.py

Path from a to h is ['a', 'b', 'd', 'h'] for a cost of 3


Actually, there is in error in this code.  Try using it to search for a goal that does not exist!